In [2]:
import ee
import geemap
import joblib
import numpy as np
import os
import threading
from threading import Lock

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

aoi = ee.Geometry.Polygon([[[-51.230662, -18.538214], [-51.230662, -18.420665], [-51.134215, -18.420665], [-51.134215, -18.538214], [-51.230662, -18.538214]]])

# Sentinel-2 collection
sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2020-01-01', '2020-04-01') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(lambda image: image.clip(aoi))

# Select bands of interest and get the median image
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
image = sentinel2.select(bands).median()

# Get the date of the middle image in the collection
middle_date = sentinel2.sort('system:time_start').toList(sentinel2.size()).get(sentinel2.size().divide(2).floor())
middle_date = ee.Image(middle_date).date()

# Set the system:time_start property for the median image
image = image.set('system:time_start', middle_date.millis())

# Apply water mask using NDWI index with a higher threshold
NDWI = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
water_mask = NDWI.gt(0.3)

# Calculate indices on the full image
MNDWI = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
NDTI = image.normalizedDifference(['B3', 'B12']).rename('NDTI')
SBI = image.expression(
    '(RED + GREEN) / BLUE',
    {
        'RED': image.select('B4'),
        'GREEN': image.select('B3'),
        'BLUE': image.select('B2')
    }
).rename('SBI')
NDSSI = image.normalizedDifference(['B11', 'B4']).rename('NDSSI')

# Add month and season
date = ee.Date(image.get('system:time_start'))
month = ee.Image.constant(date.get('month')).rename('Month')
season = ee.Image.constant(date.get('month').add(2).divide(3).floor().add(1)).rename('Season')

# Combine all bands and indices
image_with_indices = image.addBands([NDWI, MNDWI, NDTI, SBI, NDSSI, month, season])

# Load the trained model
model = joblib.load('models/LinearRegression_transparencia_model.sav')

# Extract the final estimator from the pipeline
final_estimator = model.named_steps['regressor']

# Extract coefficients and intercept from the model
coefficients = final_estimator.coef_
intercept = final_estimator.intercept_

# Normalization parameters (you may need to adjust these based on your training data)
median = 3.92  # median of training data
iqr = 2.52  # interquartile range (IQR) of test data

# Earth Engine constants
median_ee = ee.Number(median)
iqr_ee = ee.Number(iqr)

# Define water transparency prediction expression
feature_names = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'NDWI', 'MNDWI', 'NDTI', 'SBI', 'NDSSI', 'Month', 'Season']
expressions = [image_with_indices.select(name).subtract(median_ee).divide(iqr_ee).multiply(coef) for name, coef in zip(feature_names, coefficients)]

# Water transparency prediction image
predicted_image = ee.Image.constant(intercept).add(ee.Image.cat(expressions).reduce(ee.Reducer.sum())).rename('transparency_pred')

# Apply the water mask to the predicted image
masked_predicted_image = predicted_image.updateMask(water_mask)

# Set the system:time_start property for the masked predicted image
masked_predicted_image = masked_predicted_image.set('system:time_start', image.get('system:time_start'))

# Directory to save TIFF files
save_directory = 'analises_transparencia/cacu'
os.makedirs(save_directory, exist_ok=True)

# Function to split AOI and export images in parallel
def split_aoi_and_export(aoi, n_tiles, scale, image, lock, tile_list):
    aoi_bounds = aoi.bounds().coordinates().getInfo()[0]
    xmin, ymin = aoi_bounds[0][0], aoi_bounds[0][1]
    xmax, ymax = aoi_bounds[2][0], aoi_bounds[2][1]
    x_step = (xmax - xmin) / n_tiles
    y_step = (ymax - ymin) / n_tiles

    def export_tile(i, j):
        x0 = xmin + i * x_step
        x1 = xmin + (i + 1) * x_step
        y0 = ymin + j * y_step
        y1 = ymin + (j + 1) * y_step
        tile = ee.Geometry.Polygon([[[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]])
        tile_image = image.clip(tile)
        tile_list.append(tile_image)
        out_file = os.path.join(save_directory, f'PredictedTransparency_Tile_{i+1}_{j+1}.tif')
        lock.acquire()
        try:
            geemap.ee_export_image(tile_image, filename=out_file, scale=scale, region=tile)
            print(f'Image saved locally: {out_file}')
        except Exception as e:
            print(f"Error exporting tile {i+1}_{j+1}: {str(e)}")
        finally:
            lock.release()

    threads = []
    for i in range(n_tiles):
        for j in range(n_tiles):
            t = threading.Thread(target=export_tile, args=(i, j))
            threads.append(t)
            t.start()

    for t in threads:
        t.join()

# Export tiles
tile_list = []
n_tiles = 2  # NxN grid
lock = Lock()
split_aoi_and_export(aoi, n_tiles, scale=30, image=masked_predicted_image, lock=lock, tile_list=tile_list)

# Merge tiles using mosaic
merged_image = ee.ImageCollection(tile_list).mosaic()

# Calculate min and max values of water transparency after merging
min_max_values = merged_image.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

min_value = min_max_values['transparency_pred_min']
max_value = min_max_values['transparency_pred_max']

print(f"Minimum Water Transparency Value: {min_value}")
print(f"Maximum Water Transparency Value: {max_value}")

# Display the merged image on the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Visualization parameters
vis_params = {
    'min': min_value,
    'max': max_value,
    'palette': [
        '#000000', '#252525', '#525252', '#737373', '#969696', '#bdbdbd',
        '#d9d9d9', '#f0f0f0', '#ffffff', '#ffffcc', '#ffeda0', '#fed976',
        '#feb24c', '#fd8d3c', '#fc4e2a', '#e31a1c', '#bd0026', '#800026'
    ]
}
Map.addLayer(merged_image, vis_params, 'Predicted Water Transparency')
Map.addLayer(aoi, {}, 'AOI Boundary')
Map.addLayerControl()

# Function to add legend
def add_legend(map_obj, title, palette, min_value, max_value):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>low</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>high</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

# Add legend to the map
add_legend(Map, 'Predicted Water Transparency', vis_params['palette'], min_value, max_value)

# Show the map
Map

Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_1_1.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_1_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_1_2.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_1_2.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_2_1.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_2_1.tif
Generating URL ...
Please wait ...
Data downloaded to e:\Projetos\main\analises_transparencia\cacu\PredictedTransparency_Tile_2_2.tif
Image saved locally: analises_transparencia/cacu\PredictedTransparency_Tile_2_2.tif
Minimum Water Transparency Value: -1055.9283664319435
Maximum Water Transparency Value: 2024.8941273243227


Map(center=[-18.479438884362004, -51.18243850000118], controls=(WidgetControl(options=['position', 'transparen…